PDE:
$$
\begin{aligned}
&\frac{\partial^2 u}{\partial x^2}-\frac{\partial^4 u}{\partial y^4}=\left(2-x^2\right) e^{-y}\\
&\begin{aligned} 
\end{aligned}
\end{aligned}
$$
IC/BC:
$$
\begin{aligned}
&\begin{aligned} 
u_{y y}(x, 0) & =x^2 \\
u_{y y}(x, 1) & =\frac{x^2}{e} \\
u(x, 0) & =x^2 \\
u(x, 1) & =\frac{x^2}{e} \\
u(0, y) & =0 \\
u(1, y) & =e^{-y}
\end{aligned}
\end{aligned}
$$
Analytical Solution:
$$
y=x^{2} \cdot e^{-y}
$$

In [152]:
import torch

epoches=1500
h_test=1000
N_inner_point=1000
N_boundary_point=100
N_pde_point=1000

### 设置随机种子

In [153]:
def initial_seed(seed):
    torch.manual_seed(seed=seed)
    torch.backends.cudnn.deterministic=True
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed=seed)
initial_seed(8080);

### 内点

In [154]:
def inner(n=N_inner_point):
    x=torch.rand(n,1).requires_grad_(True)   #生成n*1维的0-1均匀分布随机点
    y=torch.rand(n,1).requires_grad_(True)
    cond=(2-x**2)*torch.exp(-y)
    return x,y,cond

### 内点真实解

In [155]:
def inner_analytical_solution(n=N_pde_point):
    x=torch.rand(n,1).requires_grad_(True)
    y=torch.rand(n,1).requires_grad_(True)
    cond=(x**2)*torch.exp(-y)
    return x,y,cond

### 边界点

In [156]:
def u_yy_down(n=N_boundary_point):
     x=torch.rand(n,1).requires_grad_(True)
     y=torch.zeros_like(x).requires_grad_(True)
     cond=x**2
     return x,y,cond

def u_yy_upper(n=N_boundary_point):
    x=torch.rand(n,1).requires_grad_(True)
    y=torch.ones_like(x).requires_grad_(True)
    cond=x**2/torch.e
    return x,y,cond

def u_down(n=N_boundary_point):
     x=torch.rand(n,1).requires_grad_(True)
     y=torch.zeros_like(x).requires_grad_(True)
     cond=x**2
     return x,y,cond

def u_upper(n=N_boundary_point):
    x=torch.rand(n,1).requires_grad_(True)
    y=torch.ones_like(x).requires_grad_(True)
    cond=x**2/torch.e
    return x,y,cond

def u_left(n=N_boundary_point):
     y=torch.rand(n,1).requires_grad_(True)
     x=torch.zeros_like(y).requires_grad_(True)
     cond=torch.zeros_like(x).requires_grad_(True)
     return x,y,cond

def u_right(n=N_boundary_point):
     y=torch.rand(n,1).requires_grad_(True)
     x=torch.ones_like(y).requires_grad_(True)
     cond=torch.exp(-y)
     return x,y,cond

### MLP全连接神经网络
##### 2维输入，64维隐藏层，1维输出

In [157]:
class MLP(torch.nn.Module):
    def __init__(self):
        super(MLP,self).__init__()
        self.net=torch.nn.Sequential(
            torch.nn.Linear(2,32),
            torch.nn.Tanh(),
            torch.nn.Linear(32,64),
            torch.nn.Tanh(),
            torch.nn.Linear(64,64),
            torch.nn.Tanh(),
            torch.nn.Linear(64,64),
            torch.nn.Tanh(),
            torch.nn.Linear(64,32),
            torch.nn.Tanh(),
            torch.nn.Linear(32,1)
        )
    
    def forward(self,x):
        return self.net(x)

### MSE损失、高阶求导
#### ???[grad\_outputs](https://blog.csdn.net/waitingwinter/article/details/105774720)???
先假设 $x, y$ 为一维向量, 即可设自变量因变量分别为 $\mathbf{x}=\left(x_1, x_2, \cdots, x_n\right) \in \mathbb{R}^n, y=f(\mathbf{x})=\left(y_1, y_2, \cdots, y_m\right) \in \mathbb{R}^m$ ，其对应的 Jacobian 矩阵为
$$
J=\left(\begin{array}{cccc}
\frac{\partial y_1}{\partial x_1} & \frac{\partial y_1}{\partial x_2} & \cdots & \frac{\partial y_1}{\partial x_n} \\
\frac{\partial y_2}{\partial x_1} & \frac{\partial y_2}{\partial x_2} & \cdots & \frac{\partial y_2}{\partial x_n} \\
\vdots & \vdots & \ddots & \vdots \\
\frac{\partial y_m}{\partial x_1} & \frac{\partial y_m}{\partial x_2} & \cdots & \frac{\partial y_m}{\partial x_n}
\end{array}\right)
$$
grad_outputs 是一个shape 与 outputs 一致的向量, 即
$$
{grad\_outputs}=(\begin{array}{llll}
a_{11} & a_{12} & \cdots & a_{1 m}
\end{array})^T,
$$
在给定grad_outputs 之后，真正返回的梯度为
$$
grad=\left(\begin{array}{c}
a_1 \frac{\partial y_1}{\partial x_1}+a_2 \frac{\partial y_2}{\partial x_1}+\cdots+a_m \frac{\partial y_m}{\partial x_1} \\
a_1 \frac{\partial y_1}{\partial x_2}+a_2 \frac{\partial y_2}{\partial x_2}+\cdots+a_m \frac{\partial y_m}{\partial x_2} \\
\cdots \cdots \cdots \cdots \\
a_1 \frac{\partial y_1}{\partial x_n} + a_2 \frac{\partial y_2}{\partial x_n} + \cdots + a_m \frac{\partial y_m}{\partial x_n}
\end{array}\right) \in \mathbb{R}^n
$$
为方便下文叙述我们引入记号 grad $=J \otimes g r a d \_o u t p u t s$.
其次假设 $x=\left(x_1, \cdots, x_n\right) \in \mathbb{R}^n, \mathbf{y}=\left(\mathbf{y}_1, \cdots, \mathbf{y}_{\mathbf{t}}\right) \in \mathbb{R}^{s \times t}$, 第 $\mathrm{i}$ 个列向量对应的Jacobi矩阵为
$$
J_i, 1 \leq i \leq t
$$
此时的grad_outputs 为(维度与outputs一致)
$$
{ grad\_outputs }=\left(g o_1, \cdots, g o_t\right) \in \mathbb{R}^{s \times t}
$$
由第一种情况，我们有
$$
grad=\sum_{i=1}^t J_i \otimes g o_i
$$

In [158]:
#均方误差损失函数
loss=torch.nn.MSELoss()

#高阶递归求导
def gradients(u,x,derivative_order=1):
    if derivative_order==1:
        #grad_outputs:待求导函数为矢量时需要添加的一个shape与outputs一致的向量
        #create_graph:若要计算高阶导数，则必须选为True
        return torch.autograd.grad(u,x,grad_outputs=torch.ones_like(u),create_graph=True,only_inputs=True)[0]
    else:
        #求u(x,y)的order次导数 == 求u'(x,y)的(order-1)次导数
        return gradients(gradients(u,x),x,derivative_order=derivative_order-1)

### 内点损失

In [159]:
def loss_inner(u):
    x,y,cond=inner()
    u_pred=u(torch.cat([x,y],dim=1))
    return loss(gradients(u_pred,x,2)-gradients(u_pred,y,4),cond)

### Data损失

In [160]:
def loss_data(u):
    x,y,cond=inner_analytical_solution()
    u_pred=u(torch.cat([x,y],dim=1))
    return loss(u_pred,cond)

### 边界点损失

In [161]:
def loss_yy_down(u):
     x,y,cond=u_yy_down()
     u_pred=u(torch.cat([x,y],dim=1))
     return loss(gradients(u_pred,y,2),cond)

def loss_yy_upper(u):
     x,y,cond=u_yy_upper()
     u_pred=u(torch.cat([x,y],dim=1))
     return loss(gradients(u_pred,y,2),cond)

def loss_down(u):
     x,y,cond=u_down()
     u_pred=u(torch.cat([x,y],dim=1))
     return loss(u_pred,cond)

def loss_upper(u):
     x,y,cond=u_upper()
     u_pred=u(torch.cat([x,y],dim=1))
     return loss(u_pred,cond)

def loss_left(u):
     x,y,cond=u_left()
     u_pred=u(torch.cat([x,y],dim=1))
     return loss(u_pred,cond)

def loss_right(u):
     x,y,cond=u_right()
     u_pred=u(torch.cat([x,y],dim=1))
     return loss(u_pred,cond)

### 训练

In [162]:
u=MLP()
#使用Adam优化器，学习率自适应
optimer=torch.optim.Adam(params=u.parameters())

for i in range(epoches):
    optimer.zero_grad()
    training_loss=loss_inner(u)+loss_yy_down(u)+loss_yy_upper(u)+loss_down(u)+loss_upper(u)+loss_left(u)+loss_right(u)+loss_data(u)
    training_loss.backward()
    optimer.step()
    #打印进度
    if i%100==0:
        print("Traning Epoches: ",i," Training Loss: ",torch.mean(training_loss))

Traning Epoches:  0  Training Loss:  tensor(2.1144, grad_fn=<MeanBackward0>)
Traning Epoches:  100  Training Loss:  tensor(0.2069, grad_fn=<MeanBackward0>)
Traning Epoches:  200  Training Loss:  tensor(0.0518, grad_fn=<MeanBackward0>)
Traning Epoches:  300  Training Loss:  tensor(0.0232, grad_fn=<MeanBackward0>)
Traning Epoches:  400  Training Loss:  tensor(0.0127, grad_fn=<MeanBackward0>)
Traning Epoches:  500  Training Loss:  tensor(0.0054, grad_fn=<MeanBackward0>)
Traning Epoches:  600  Training Loss:  tensor(0.0020, grad_fn=<MeanBackward0>)
Traning Epoches:  700  Training Loss:  tensor(0.0008, grad_fn=<MeanBackward0>)
Traning Epoches:  800  Training Loss:  tensor(0.0014, grad_fn=<MeanBackward0>)
Traning Epoches:  900  Training Loss:  tensor(0.0004, grad_fn=<MeanBackward0>)
Traning Epoches:  1000  Training Loss:  tensor(0.0002, grad_fn=<MeanBackward0>)
Traning Epoches:  1100  Training Loss:  tensor(0.0002, grad_fn=<MeanBackward0>)
Traning Epoches:  1200  Training Loss:  tensor(0.000

### 预测并检验效果

In [163]:
#生成0-1均匀分布的点
x_uniform=torch.linspace(0,1,h_test)
#生成二维网格
xx,yy=torch.meshgrid(x_uniform,x_uniform)
xx,yy=xx.reshape(-1,1),yy.reshape(-1,1)
#将1*n的横坐标集(x1,x2,...,xn)和纵坐标集(y1,y2,...,yn)
#先变为{(x1),(x2),...,(xn)}和{(y1),(y2),...,(yn)}，即n*1
#再变为网格点{(x1,y1),(x2,y2),...,(xn,yn)}的序列，即n*2
xy=torch.cat([xx,yy],dim=1)
xy_pred=u(xy)

#误差
xy_real=xx**2*torch.exp(-yy)
error=torch.abs(xy_pred-xy_real)
print("Max error is ",float(torch.max(error)))
print("Average error is ",float(torch.mean(error)))
print("Pred: \n",xy_pred)
print("Real: \n",xy_real)
print("Error: \n",error)


Max error is  0.024331748485565186
Average error is  0.016554679721593857
Pred: 
 tensor([[-0.0161],
        [-0.0161],
        [-0.0161],
        ...,
        [ 0.3503],
        [ 0.3500],
        [ 0.3496]], grad_fn=<AddmmBackward0>)
Real: 
 tensor([[0.0000],
        [0.0000],
        [0.0000],
        ...,
        [0.3686],
        [0.3682],
        [0.3679]])
Error: 
 tensor([[0.0161],
        [0.0161],
        [0.0161],
        ...,
        [0.0183],
        [0.0183],
        [0.0182]], grad_fn=<AbsBackward0>)
